In [10]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
from sklearn.model_selection import train_test_split

/Users/varun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
img_dir = os.path.join('food-101', 'images')
meta_dir = os.path.join('food-101', 'meta')
train_json_p = os.path.join(meta_dir, 'train.json')
test_json_p = os.path.join(meta_dir, 'test.json')
label_txt_p = os.path.join(meta_dir, 'labels.txt')
class_txt_p = os.path.join(meta_dir, 'classes.txt')
class_names = []
with open(class_txt_p, 'r') as f:
    class_names = sorted([line.strip() for line in f.readlines()])
num_classes = len(class_names)
print(f'Number of classes: {num_classes}')





Number of classes: 101


In [12]:
def parse_json_to_list(json_p):
    with open(json_p, 'r') as f:
        data = json.load(f)
    
    file_label_list = []
    for food_label, rel_paths in data.items():
        for rel_path in rel_paths:
            full_img_paths = os.path.join(img_dir, rel_path + '.jpg')
            file_label_list.append({'filepath': full_img_paths, 'label': food_label})
        file_label_list.append({'filepath':full_img_paths, 'label':food_label})
    return file_label_list
print(f'Parsing {train_json_p}...')
train_file_label_list = parse_json_to_list(train_json_p)
print(f'Parsing {test_json_p}...')
test_file_label_list = parse_json_to_list(test_json_p)

Parsing food-101/meta/train.json...
Parsing food-101/meta/test.json...


In [13]:
train_df = pd.DataFrame(train_file_label_list)
test_df = pd.DataFrame(test_file_label_list)

In [14]:
len(train_df), len(test_df)

(75851, 25351)

In [15]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)
len(train_df), len(val_df), len(test_df)

(60680, 15171, 25351)

In [16]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_data_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_data_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    classes=class_names
)
val_generator = val_data_gen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=class_names
)

Found 60680 validated image filenames belonging to 101 classes.
Found 15171 validated image filenames belonging to 101 classes.


In [17]:
from tensorflow.keras import layers, models
def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)  # Define input_shape based on IMG_SIZE
model = create_model(input_shape, num_classes)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        13,029 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,181,989 (42.66 MB)

 Trainable params: 11,181,989 (42.66 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
print(f"train_generator.samples (total images): {train_generator.samples}")
print(f"train_generator.n (total images, alternative): {train_generator.n}") # Should be the same as .samples
print(f"train_generator.batch_size: {train_generator.batch_size}")
print(f"train_generator.num_classes: {len(train_generator.class_indices)}")

# Ensure the generator has actually been initialized and can provide data
# Try to draw a single batch manually
try:
    first_batch_images, first_batch_labels = next(train_generator)
    print(f"Successfully drew one batch. Images shape: {first_batch_images.shape}, Labels shape: {first_batch_labels.shape}")
except StopIteration:
    print("Error: train_generator is empty when trying to draw a batch!")
except Exception as e:
    print(f"Error drawing batch: {e}")


train_generator.samples (total images): 60680
train_generator.n (total images, alternative): 60680
train_generator.batch_size: 32
train_generator.num_classes: 101
Successfully drew one batch. Images shape: (32, 224, 224, 3), Labels shape: (32, 101)


In [20]:
from tensorflow.keras.applications import MobileNetV2 # Or ResNet50, EfficientNetB0 etc.
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras import Model

# Assuming IMG_SIZE and num_classes are defined from your data loading
# e.g., IMG_SIZE = (224, 224), num_classes = 101

# 1. Load the Pre-trained Base Model
#    - input_shape: Must match your image dimensions (e.g., (224, 224, 3))
#    - include_top=False: Crucial! Excludes the original ImageNet classification head.
#    - weights='imagenet': Loads the weights pre-trained on ImageNet.
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)

# 2. Freeze the convolutional base
#    This makes its layers non-trainable, preserving the pre-learned features.
base_model.trainable = False

# 3. Add Your Custom Classification Head on top of the base model
x = base_model.output # Get the output tensor from the pre-trained base
x = Flatten()(x)      # Flatten the 3D output to 1D for Dense layers
x = Dense(128, activation='relu')(x) # A new hidden Dense layer
x = Dropout(0.5)(x)   # Dropout for regularization
predictions = Dense(num_classes, activation='softmax')(x) # Your final output layer for 101 classes

# 4. Create the new combined model
model = Model(inputs=base_model.input, outputs=predictions)

# 5. Compile the Model
#    Use your chosen optimizer and loss function (categorical_crossentropy for one-hot labels)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. (Optional) Print model summary to see the new architecture
model.summary()

# 7. Train the Model (same as before)
history = model.fit(
     train_generator,
     epochs=10, # Start with a few epochs, then monitor
     validation_data=val_generator
)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 10,299,301 (39.29 MB)

 Trainable params: 8,041,317 (30.68 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 414s 217ms/step - accuracy: 0.0269 - loss: 127.3734 - val_accuracy: 0.1508 - val_loss: 14.4508
Epoch 2/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 436s 230ms/step - accuracy: 0.0722 - loss: 22.9734 - val_accuracy: 0.1535 - val_loss: 9.6045
Epoch 3/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 480s 253ms/step - accuracy: 0.0800 - loss: 20.2714 - val_accuracy: 0.1488 - val_loss: 11.9292
Epoch 4/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 488s 257ms/step - accuracy: 0.0819 - loss: 21.6905 - val_accuracy: 0.1509 - val_loss: 14.2252
Epoch 5/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 558s 294ms/step - accuracy: 0.0867 - loss: 23.8218 - val_accuracy: 0.1894 - val_loss: 12.4147
Epoch 6/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 403s 213ms/step - accuracy: 0.0954 - loss: 25.5263 - val_accuracy: 0.1814 - val_loss: 14.6744
Epoch 7/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 483s 255ms/step - accuracy: 0.0996 - loss: 25.9513 - val_accuracy: 0.2064 - val_loss: 14.4889
Epoch 8/10
1897/1897 ━━━━━━━━━━━━━━━━━━━━ 422s 2

In [21]:
import json
import pandas as pd

CALORIE_DB_PATH = 'Cal.json' # Point to your new JSON file

try:
    with open(CALORIE_DB_PATH, 'r') as f:
        calorie_data = json.load(f)
    print("Calorie data loaded successfully from JSON.")

    # You can convert it to a DataFrame if you still prefer DataFrame operations
    calorie_df = pd.DataFrame.from_dict(calorie_data, orient='index')
    # If you want food_name to be a column instead of index:
    # calorie_df.reset_index(inplace=True)
    # calorie_df.rename(columns={'index': 'food_name'}, inplace=True)

except Exception as e:
    print(f"Error loading calorie data from JSON: {e}")
    calorie_data = {} # Fallback to empty dictionary
    calorie_df = pd.DataFrame() # Fallback to empty DataFrame


# Now, your get_calorie_info function would work similarly (assuming calorie_df is used):
def get_calorie_info(food_name, calorie_df):
    if calorie_df.empty:
        return {"error": "Calorie database not loaded."}
    try:
        info = calorie_df.loc[food_name].to_dict()
        return info
    except KeyError:
        return {"error": f"Calorie information not found for '{food_name}'"}

# Or if you prefer to use the dictionary directly:
def get_calorie_info_from_dict(food_name, calorie_data_dict):
    return calorie_data_dict.get(food_name, {"error": f"Calorie information not found for '{food_name}'"})

# Example usage with dictionary:
# calories = get_calorie_info_from_dict(predicted_food, calorie_data)

Calorie data loaded successfully from JSON.


In [26]:
model.save('Food_model.keras') # Or 'my_food_model.keras' for newer Keras versions
# Load the model later with:
# from tensorflow.keras.models import load_model
# model = load_model('my_food_model.h5') # Or 'my_food_model.keras'